In [ ]:
import io
from google.colab import files
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
import random
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import RepeatedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score
from sklearn.naive_bayes import GaussianNB as Naive
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import cross_val_score

### **Lendo a Base de Dados**

In [ ]:
uploaded = files.upload()

Saving BaseReduzida1_CheckpointFinal.csv to BaseReduzida1_CheckpointFinal.csv


In [ ]:
dataset = pd.read_csv(io.BytesIO(uploaded['BaseReduzida1_CheckpointFinal.csv']))

In [ ]:
dataset = pd.read_csv('BaseReduzida1_CheckpointFinal.csv')

In [ ]:
dataset.head()

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Seat comfort,Departure/Arrival time convenient,Food and drink,Gate location,Inflight wifi service,Inflight entertainment,Online support,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction_v2
0,1,1,60,1,1,3453,5,5,5,5,5,4,5,3,3,3,3,3,3,3,3,0.0,1
1,1,1,60,1,1,2761,3,3,3,3,3,4,4,5,5,5,5,4,5,3,1,0.0,1
2,0,1,29,0,2,1374,2,4,2,1,3,2,3,3,5,5,5,3,5,3,0,0.0,0
3,0,1,39,0,2,2108,3,4,3,3,5,3,5,5,5,3,5,5,4,5,0,0.0,0
4,1,1,40,1,1,2606,1,1,1,1,3,4,4,5,5,5,5,3,5,5,11,0.0,1


### **Boosting**

####**Árvore de Decisão**

In [ ]:
for estimators in [10, 15, 20]:
  acuracia = []
  for x in range(10):
    seed = random.randint(900,5000)
    X = dataset.iloc[:,:-1].values
    y = dataset.iloc[:,-1].values
 
    rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=seed)
    for train, test in rkf.split(X, y):
      X_train = X[train]
      X_test = X[test]
      y_train = y[train]
      y_test = y[test]
 
    dtc = DecisionTreeClassifier(max_depth = 7, max_leaf_nodes = 19, min_samples_split = 2)
    abc = AdaBoostClassifier(base_estimator=dtc, n_estimators=estimators, random_state=seed)
    abc.fit(X_train, y_train)
    y_pred = abc.predict(X_test)
 
    acc = accuracy_score(y_test, y_pred)
    acuracia.append(acc)
 
  print('Quantidade de classificadores:', estimators)
  print('Acurácia:', mean(acuracia))
  print('DP: %.3f' % std(acuracia))
  print(acuracia)

Quantidade de classificadores: 10
Acurácia: 0.9341496598639456
DP: 0.006
[0.9333333333333333, 0.9455782312925171, 0.9360544217687075, 0.9333333333333333, 0.9346938775510204, 0.9442176870748299, 0.9272108843537415, 0.9299319727891157, 0.9251700680272109, 0.9319727891156463]
Quantidade de classificadores: 15
Acurácia: 0.9373469387755101
DP: 0.007
[0.9285714285714286, 0.9421768707482994, 0.926530612244898, 0.9340136054421768, 0.9435374149659864, 0.9292517006802721, 0.9455782312925171, 0.9448979591836735, 0.9346938775510204, 0.9442176870748299]
Quantidade de classificadores: 20
Acurácia: 0.9417006802721086
DP: 0.005
[0.9319727891156463, 0.9346938775510204, 0.9435374149659864, 0.9455782312925171, 0.9428571428571428, 0.9476190476190476, 0.9387755102040817, 0.9414965986394558, 0.9462585034013605, 0.9442176870748299]


####**Gaussian NB**

In [ ]:
for estimators in [10, 15, 20]:
  acuracia = []
  for x in range(10):
    seed = random.randint(900,5000)
    X = dataset.iloc[:,:-1].values
    y = dataset.iloc[:,-1].values

    rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=seed)
    for train, test in rkf.split(X, y):
      X_train = X[train]
      X_test = X[test]
      y_train = y[train]
      y_test = y[test]

    abc = AdaBoostClassifier(base_estimator=Naive(), n_estimators=estimators, random_state=seed)
    abc.fit(X_train, y_train)
    y_pred = abc.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    acuracia.append(acc)

  print('Quantidade de classificadores:', estimators)
  print('Acurácia:', mean(acuracia))
  print('DP: %.3f' % std(acuracia))
  print(acuracia)

Quantidade de classificadores: 10
Acurácia: 0.5759863945578231
DP: 0.045
[0.5571428571428572, 0.6285714285714286, 0.5462585034013605, 0.6489795918367347, 0.5503401360544218, 0.527891156462585, 0.5653061224489796, 0.64421768707483, 0.5238095238095238, 0.5673469387755102]
Quantidade de classificadores: 15
Acurácia: 0.599795918367347
DP: 0.094
[0.5510204081632653, 0.7448979591836735, 0.6625850340136055, 0.6755102040816326, 0.6843537414965987, 0.5326530612244897, 0.6476190476190476, 0.47959183673469385, 0.582312925170068, 0.43741496598639457]
Quantidade de classificadores: 20
Acurácia: 0.5270068027210885
DP: 0.074
[0.5727891156462585, 0.5448979591836735, 0.4523809523809524, 0.591156462585034, 0.427891156462585, 0.6632653061224489, 0.507482993197279, 0.5115646258503401, 0.5768707482993197, 0.4217687074829932]


####**KNN**

***Feito no weka:***

Para 10, 15 e 20 classificadores, os resultados foram os mesmos:

Média acc: 89.94087320856286

DP: 0.6918072158803398

####**MLP**

**10 classificadores**

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import load_iris
from sklearn.ensemble import AdaBoostClassifier

class customMLPClassifer(MLPClassifier):
    def resample_with_replacement(self, X_train, y_train, sample_weight):

        # normalize sample_weights if not already
        sample_weight = sample_weight / sample_weight.sum(dtype=np.float64)

        X_train_resampled = np.zeros((len(X_train), len(X_train[0])), dtype=np.float32)
        y_train_resampled = np.zeros((len(y_train)), dtype=np.int)
        for i in range(len(X_train)):
            # draw a number from 0 to len(X_train)-1
            draw = np.random.choice(np.arange(len(X_train)), p=sample_weight)

            # place the X and y at the drawn number into the resampled X and y
            X_train_resampled[i] = X_train[draw]
            y_train_resampled[i] = y_train[draw]

        return X_train_resampled, y_train_resampled


    def fit(self, X, y, sample_weight=None):
        if sample_weight is not None:
            X, y = self.resample_with_replacement(X, y, sample_weight)

        return self._fit(X, y, incremental=(self.warm_start and
                                            hasattr(self, "classes_")))

acuracia = []
for x in range(10):
  seed = random.randint(900,5000)
  X = dataset.iloc[:,:-1].values
  y = dataset.iloc[:,-1].values
  minmax = MinMaxScaler()
  X = minmax.fit_transform(X)

  rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=seed)
  for train, test in rkf.split(X, y):
    X_train = X[train]
    X_test = X[test]
    y_train = y[train]
    y_test = y[test]

  adabooster = AdaBoostClassifier(base_estimator=customMLPClassifer(solver='sgd', momentum=0.8, 
                                                                    hidden_layer_sizes=16, 
                                                                    learning_rate='constant', 
                                                                    learning_rate_init=0.1,
                                                                    max_iter=100, 
                                                                    random_state=1), n_estimators=10, random_state=seed)
  adabooster.fit(X_train, y_train)
  y_pred = adabooster.predict(X_test)
  acc = accuracy_score(y_test, y_pred)
  acuracia.append(acc)

print('Quantidade de classificadores: 10')
print('Acurácia:', mean(acuracia))
print('DP: %.3f' % std(acuracia))

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Quantidade de classificadores: 10
Acurácia: 0.9370068027210884
DP: 0.005


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [ ]:
acuracia

[0.9380952380952381,
 0.9326530612244898,
 0.9326530612244898,
 0.9489795918367347,
 0.9380952380952381,
 0.9360544217687075,
 0.9333333333333333,
 0.9401360544217687,
 0.9312925170068027,
 0.9387755102040817]

**15 Classificadores**

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import load_iris
from sklearn.ensemble import AdaBoostClassifier

class customMLPClassifer(MLPClassifier):
    def resample_with_replacement(self, X_train, y_train, sample_weight):

        # normalize sample_weights if not already
        sample_weight = sample_weight / sample_weight.sum(dtype=np.float64)

        X_train_resampled = np.zeros((len(X_train), len(X_train[0])), dtype=np.float32)
        y_train_resampled = np.zeros((len(y_train)), dtype=np.int)
        for i in range(len(X_train)):
            # draw a number from 0 to len(X_train)-1
            draw = np.random.choice(np.arange(len(X_train)), p=sample_weight)

            # place the X and y at the drawn number into the resampled X and y
            X_train_resampled[i] = X_train[draw]
            y_train_resampled[i] = y_train[draw]

        return X_train_resampled, y_train_resampled


    def fit(self, X, y, sample_weight=None):
        if sample_weight is not None:
            X, y = self.resample_with_replacement(X, y, sample_weight)

        return self._fit(X, y, incremental=(self.warm_start and
                                            hasattr(self, "classes_")))

acuracia = []
for x in range(10):
  seed = random.randint(900,5000)
  X = dataset.iloc[:,:-1].values
  y = dataset.iloc[:,-1].values
  minmax = MinMaxScaler()
  X = minmax.fit_transform(X)

  rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=seed)
  for train, test in rkf.split(X, y):
    X_train = X[train]
    X_test = X[test]
    y_train = y[train]
    y_test = y[test]

  adabooster = AdaBoostClassifier(base_estimator=customMLPClassifer(solver='sgd', momentum=0.8, 
                                                                    hidden_layer_sizes=16, 
                                                                    learning_rate='constant', 
                                                                    learning_rate_init=0.1,
                                                                    max_iter=100, 
                                                                    random_state=1), n_estimators=15, random_state=seed)
  adabooster.fit(X_train, y_train)
  y_pred = adabooster.predict(X_test)
  acc = accuracy_score(y_test, y_pred)
  acuracia.append(acc)

print('Quantidade de classificadores: 15')
print('Acurácia:', mean(acuracia))
print('DP: %.3f' % std(acuracia))

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Quantidade de classificadores: 15
Acurácia: 0.9408163265306124
DP: 0.009


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [ ]:
acuracia

[0.9530612244897959,
 0.9510204081632653,
 0.9340136054421768,
 0.9387755102040817,
 0.935374149659864,
 0.9394557823129251,
 0.9224489795918367,
 0.9421768707482994,
 0.9469387755102041,
 0.9448979591836735]

**20 Classificadores**

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import load_iris
from sklearn.ensemble import AdaBoostClassifier

class customMLPClassifer(MLPClassifier):
    def resample_with_replacement(self, X_train, y_train, sample_weight):

        # normalize sample_weights if not already
        sample_weight = sample_weight / sample_weight.sum(dtype=np.float64)

        X_train_resampled = np.zeros((len(X_train), len(X_train[0])), dtype=np.float32)
        y_train_resampled = np.zeros((len(y_train)), dtype=np.int)
        for i in range(len(X_train)):
            # draw a number from 0 to len(X_train)-1
            draw = np.random.choice(np.arange(len(X_train)), p=sample_weight)

            # place the X and y at the drawn number into the resampled X and y
            X_train_resampled[i] = X_train[draw]
            y_train_resampled[i] = y_train[draw]

        return X_train_resampled, y_train_resampled


    def fit(self, X, y, sample_weight=None):
        if sample_weight is not None:
            X, y = self.resample_with_replacement(X, y, sample_weight)

        return self._fit(X, y, incremental=(self.warm_start and
                                            hasattr(self, "classes_")))

acuracia = []
for x in range(10):
  seed = random.randint(900,5000)
  X = dataset.iloc[:,:-1].values
  y = dataset.iloc[:,-1].values
  minmax = MinMaxScaler()
  X = minmax.fit_transform(X)

  rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=seed)
  for train, test in rkf.split(X, y):
    X_train = X[train]
    X_test = X[test]
    y_train = y[train]
    y_test = y[test]

  adabooster = AdaBoostClassifier(base_estimator=customMLPClassifer(solver='sgd', momentum=0.8, 
                                                                    hidden_layer_sizes=16, 
                                                                    learning_rate='constant', 
                                                                    learning_rate_init=0.1,
                                                                    max_iter=100, 
                                                                    random_state=1), n_estimators=20, random_state=seed)
  adabooster.fit(X_train, y_train)
  y_pred = adabooster.predict(X_test)
  acc = accuracy_score(y_test, y_pred)
  acuracia.append(acc)

print('Quantidade de classificadores: 20')
print('Acurácia:', mean(acuracia))
print('DP: %.3f' % std(acuracia))

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Quantidade de classificadores: 20
Acurácia: 0.938095238095238
DP: 0.006


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [ ]:
acuracia

[0.9476190476190476,
 0.9435374149659864,
 0.936734693877551,
 0.9448979591836735,
 0.9258503401360544,
 0.9319727891156463,
 0.9387755102040817,
 0.9394557823129251,
 0.9401360544217687,
 0.9319727891156463]

### **Bagging**

####**Árvore de Decisão**

In [ ]:
for estimators in [10, 15, 20]:
  acuracia = []
  for x in range(10):
    seed = random.randint(900,5000)
    X = dataset.iloc[:,:-1].values
    y = dataset.iloc[:,-1].values

    rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=seed)
    for train, test in rkf.split(X, y):
      X_train = X[train]
      X_test = X[test]
      y_train = y[train]
      y_test = y[test]

    dtc = DecisionTreeClassifier(max_depth = 7, max_leaf_nodes = 19, min_samples_split = 2)
    bc = BaggingClassifier(base_estimator=dtc, n_estimators=estimators, random_state=seed)
    bc.fit(X_train, y_train)
    y_pred = bc.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    acuracia.append(acc)

  print('Quantidade de classificadores:', estimators)
  print('Acurácia:', mean(acuracia))
  print('DP: %.3f' % std(acuracia))
  print(acuracia)

Quantidade de classificadores: 10
Acurácia: 0.9015646258503403
DP: 0.005
[0.9054421768707483, 0.9, 0.8952380952380953, 0.9013605442176871, 0.9006802721088435, 0.9047619047619048, 0.9054421768707483, 0.8925170068027211, 0.9108843537414966, 0.8993197278911564]
Quantidade de classificadores: 15
Acurácia: 0.901360544217687
DP: 0.006
[0.9115646258503401, 0.8972789115646258, 0.8938775510204081, 0.898639455782313, 0.9020408163265307, 0.9095238095238095, 0.8972789115646258, 0.8945578231292517, 0.9013605442176871, 0.9074829931972789]
Quantidade de classificadores: 20
Acurácia: 0.8982312925170068
DP: 0.005
[0.8965986394557823, 0.8904761904761904, 0.8972789115646258, 0.891156462585034, 0.8959183673469387, 0.8979591836734694, 0.9020408163265307, 0.9006802721088435, 0.9047619047619048, 0.9054421768707483]


####**KNN**

In [ ]:
for estimators in [10, 15, 20]:
  acuracia = []
  for x in range(10):
    seed = random.randint(900,5000)
    X = dataset.iloc[:,:-1].values
    y = dataset.iloc[:,-1].values
    minmax = MinMaxScaler()
    X = minmax.fit_transform(X)

    rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=seed)
    for train, test in rkf.split(X, y):
      X_train = X[train]
      X_test = X[test]
      y_train = y[train]
      y_test = y[test]

    knn = KNeighborsClassifier(n_neighbors=7, metric='euclidean')
    bc = BaggingClassifier(base_estimator=knn, n_estimators=estimators, random_state=seed)
    bc.fit(X_train, y_train)
    y_pred = bc.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    acuracia.append(acc)

  print('Quantidade de classificadores:', estimators)
  print('Acurácia:', mean(acuracia))
  print('DP: %.3f' % std(acuracia))
  print(acuracia)

Quantidade de classificadores: 10
Acurácia: 0.9093197278911566
DP: 0.004
[0.9047619047619048, 0.9108843537414966, 0.9081632653061225, 0.9163265306122449, 0.9081632653061225, 0.9081632653061225, 0.9068027210884354, 0.9102040816326531, 0.9054421768707483, 0.9142857142857143]
Quantidade de classificadores: 15
Acurácia: 0.9136734693877552
DP: 0.006
[0.917687074829932, 0.9108843537414966, 0.9163265306122449, 0.9244897959183673, 0.9122448979591836, 0.917687074829932, 0.9108843537414966, 0.9136054421768708, 0.9102040816326531, 0.9027210884353741]
Quantidade de classificadores: 20
Acurácia: 0.9143537414965988
DP: 0.007
[0.919047619047619, 0.919047619047619, 0.9217687074829932, 0.9149659863945578, 0.9210884353741496, 0.9149659863945578, 0.8965986394557823, 0.9142857142857143, 0.9163265306122449, 0.9054421768707483]


####**Gaussian NB**

In [ ]:
for estimators in [10, 15, 20]:
  acuracia = []
  for x in range(10):
    seed = random.randint(900,5000)
    X = dataset.iloc[:,:-1].values
    y = dataset.iloc[:,-1].values

    rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=seed)
    for train, test in rkf.split(X, y):
      X_train = X[train]
      X_test = X[test]
      y_train = y[train]
      y_test = y[test]

    bc = BaggingClassifier(base_estimator=Naive(), n_estimators=estimators, random_state=seed)
    bc.fit(X_train, y_train)
    y_pred = bc.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    acuracia.append(acc)

  print('Quantidade de classificadores:', estimators)
  print('Acurácia:', mean(acuracia))
  print('DP: %.3f' % std(acuracia))
  print(acuracia)

Quantidade de classificadores: 10
Acurácia: 0.8216326530612245
DP: 0.006
[0.8149659863945579, 0.8238095238095238, 0.8142857142857143, 0.8210884353741497, 0.8156462585034013, 0.8353741496598639, 0.8272108843537415, 0.8197278911564626, 0.8170068027210884, 0.8272108843537415]
Quantidade de classificadores: 15
Acurácia: 0.8195918367346939
DP: 0.011
[0.826530612244898, 0.8312925170068027, 0.8306122448979592, 0.7979591836734694, 0.8163265306122449, 0.8047619047619048, 0.8197278911564626, 0.8129251700680272, 0.827891156462585, 0.827891156462585]
Quantidade de classificadores: 20
Acurácia: 0.8183673469387756
DP: 0.012
[0.8013605442176871, 0.8204081632653061, 0.826530612244898, 0.8156462585034013, 0.8306122448979592, 0.7993197278911565, 0.8299319727891157, 0.8061224489795918, 0.8183673469387756, 0.8353741496598639]


####**MLP**

In [ ]:
for estimators in [10, 15, 20]:
  acuracia = []
  for x in range(10):
    seed = random.randint(900,5000)
    X = dataset.iloc[:,:-1].values
    y = dataset.iloc[:,-1].values
    minmax = MinMaxScaler()
    X = minmax.fit_transform(X)

    rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=seed)
    for train, test in rkf.split(X, y):
      X_train = X[train]
      X_test = X[test]
      y_train = y[train]
      y_test = y[test]

    mlp = MLPClassifier(solver='sgd', momentum=0.8, 
                              hidden_layer_sizes=16, 
                              learning_rate='constant', 
                              learning_rate_init=0.1,
                              max_iter=100, 
                              random_state=1)
    
    bc = BaggingClassifier(base_estimator=mlp, n_estimators=estimators, random_state=seed)
    bc.fit(X_train, y_train)
    y_pred = bc.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    acuracia.append(acc)

  print('Quantidade de classificadores:', estimators)
  print('Acurácia:', mean(acuracia))
  print('DP: %.3f' % std(acuracia))
  print(acuracia)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Quantidade de classificadores: 10
Acurácia: 0.9297278911564627
DP: 0.006
[0.9326530612244898, 0.9346938775510204, 0.935374149659864, 0.936734693877551, 0.9231292517006803, 0.926530612244898, 0.9380952380952381, 0.9217687074829932, 0.9224489795918367, 0.9258503401360544]


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Quantidade de classificadores: 15
Acurácia: 0.9310204081632654
DP: 0.004
[0.9258503401360544, 0.927891156462585, 0.9340136054421768, 0.935374149659864, 0.9285714285714286, 0.9387755102040817, 0.9326530612244898, 0.9272108843537415, 0.9319727891156463, 0.927891156462585]


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Quantidade de classificadores: 20
Acurácia: 0.9334013605442179
DP: 0.006
[0.9312925170068027, 0.9183673469387755, 0.9374149659863945, 0.9319727891156463, 0.9374149659863945, 0.9394557823129251, 0.9380952380952381, 0.9319727891156463, 0.9306122448979591, 0.9374149659863945]


**Resultados:**

**Quantidade de classificadores: 10**

Acurácia: 0.9297278911564627

DP: 0.006

[0.9326530612244898, 0.9346938775510204, 0.935374149659864, 0.936734693877551, 0.9231292517006803, 0.926530612244898, 0.9380952380952381, 0.9217687074829932, 0.9224489795918367, 0.9258503401360544]

**Quantidade de classificadores: 15**

Acurácia: 0.9310204081632654

DP: 0.004

[0.9258503401360544, 0.927891156462585, 0.9340136054421768, 0.935374149659864, 0.9285714285714286, 0.9387755102040817, 0.9326530612244898, 0.9272108843537415, 0.9319727891156463, 0.927891156462585]

**Quantidade de classificadores: 20**

Acurácia: 0.9334013605442179

DP: 0.006

[0.9312925170068027, 0.9183673469387755, 0.9374149659863945, 0.9319727891156463, 0.9374149659863945, 0.9394557823129251, 0.9380952380952381, 0.9319727891156463, 0.9306122448979591, 0.9374149659863945]


### **Stacking Homogêneo**

 Para o Stacking homogêneo, as mesmas configurações do Bagging e Boosting são utilizadas (tamanhos 10, 15 e 20 e usando AD, k-NN, NB e MLP), usando um combinador que tenha bom desempenho para a sua base de dados (teste uns 2 ou 3 e veja qual seria o melhor).

####**Árvore de Decisão**

In [ ]:
for tam in [10, 15, 20]:
  acuracia = []
  for x in range(10):
    seed = random.randint(900,5000)
    X = dataset.iloc[:,:-1].values
    y = dataset.iloc[:,-1].values

    rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=seed)
    for train, test in rkf.split(X, y):
      X_train = X[train]
      X_test = X[test]
      y_train = y[train]
      y_test = y[test]

    dtc = DecisionTreeClassifier(max_depth = 7, max_leaf_nodes = 19, min_samples_split = 2)

    if(tam == 10): 
      estimators = [('dtc1', dtc), ('dtc2', dtc), ('dtc3', dtc), ('dtc4', dtc), ('dtc5', dtc),
                    ('dtc6', dtc), ('dtc7', dtc), ('dtc8', dtc), ('dtc9', dtc), ('dtc10', dtc)]

    elif(tam == 15):
      estimators = [('dtc1', dtc), ('dtc2', dtc), ('dtc3', dtc), ('dtc4', dtc), ('dtc5', dtc),
                    ('dtc6', dtc), ('dtc7', dtc), ('dtc8', dtc), ('dtc9', dtc), ('dtc10', dtc),
                    ('dtc11', dtc), ('dtc12', dtc), ('dtc13', dtc), ('dtc14', dtc), ('dtc15', dtc)]
    
    elif(tam == 20):
      estimators = [('dtc1', dtc), ('dtc2', dtc), ('dtc3', dtc), ('dtc4', dtc), ('dtc5', dtc),
                    ('dtc6', dtc), ('dtc7', dtc), ('dtc8', dtc), ('dtc9', dtc), ('dtc10', dtc),
                    ('dtc11', dtc), ('dtc12', dtc), ('dtc13', dtc), ('dtc14', dtc), ('dtc15', dtc),
                    ('dtc16', dtc), ('dtc17', dtc), ('dtc18', dtc), ('dtc19', dtc), ('dtc20', dtc)]

    sclf = StackingClassifier(estimators=estimators, final_estimator=DecisionTreeClassifier())
    sclf.fit(X_train, y_train)
    y_pred = sclf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    acuracia.append(acc)

  print('Quantidade de classificadores:', tam)
  print('Acurácia:', mean(acuracia))
  print('DP: %.3f' % std(acuracia))
  print(acuracia)

Quantidade de classificadores: 10
Acurácia: 0.8891156462585034
DP: 0.010
[0.8918367346938776, 0.8741496598639455, 0.8945578231292517, 0.8775510204081632, 0.898639455782313, 0.8904761904761904, 0.8734693877551021, 0.8918367346938776, 0.9020408163265307, 0.8965986394557823]
Quantidade de classificadores: 15
Acurácia: 0.8919727891156463
DP: 0.010
[0.8945578231292517, 0.8816326530612245, 0.9013605442176871, 0.8857142857142857, 0.8891156462585034, 0.9061224489795918, 0.8891156462585034, 0.8918367346938776, 0.8748299319727891, 0.9054421768707483]


KeyboardInterrupt: ignored

In [ ]:
for tam in [10, 15, 20]:
  acuracia = []
  for x in range(10):
    seed = random.randint(900,5000)
    X = dataset.iloc[:,:-1].values
    y = dataset.iloc[:,-1].values

    rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=seed)
    for train, test in rkf.split(X, y):
      X_train = X[train]
      X_test = X[test]
      y_train = y[train]
      y_test = y[test]

    dtc = DecisionTreeClassifier(max_depth = 7, max_leaf_nodes = 19, min_samples_split = 2)

    if(tam == 10): 
      estimators = [('dtc1', dtc), ('dtc2', dtc), ('dtc3', dtc), ('dtc4', dtc), ('dtc5', dtc),
                    ('dtc6', dtc), ('dtc7', dtc), ('dtc8', dtc), ('dtc9', dtc), ('dtc10', dtc)]

    elif(tam == 15):
      estimators = [('dtc1', dtc), ('dtc2', dtc), ('dtc3', dtc), ('dtc4', dtc), ('dtc5', dtc),
                    ('dtc6', dtc), ('dtc7', dtc), ('dtc8', dtc), ('dtc9', dtc), ('dtc10', dtc),
                    ('dtc11', dtc), ('dtc12', dtc), ('dtc13', dtc), ('dtc14', dtc), ('dtc15', dtc)]
    
    elif(tam == 20):
      estimators = [('dtc1', dtc), ('dtc2', dtc), ('dtc3', dtc), ('dtc4', dtc), ('dtc5', dtc),
                    ('dtc6', dtc), ('dtc7', dtc), ('dtc8', dtc), ('dtc9', dtc), ('dtc10', dtc),
                    ('dtc11', dtc), ('dtc12', dtc), ('dtc13', dtc), ('dtc14', dtc), ('dtc15', dtc),
                    ('dtc16', dtc), ('dtc17', dtc), ('dtc18', dtc), ('dtc19', dtc), ('dtc20', dtc)]

    sclf = StackingClassifier(estimators=estimators, final_estimator=Naive())
    sclf.fit(X_train, y_train)
    y_pred = sclf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    acuracia.append(acc)

  print('Quantidade de classificadores:', tam)
  print('Acurácia:', mean(acuracia))
  print('DP: %.3f' % std(acuracia))
  print(acuracia)

Quantidade de classificadores: 10
Acurácia: 0.8932653061224489
DP: 0.007
[0.8863945578231293, 0.8918367346938776, 0.8952380952380953, 0.9061224489795918, 0.8884353741496599, 0.898639455782313, 0.9020408163265307, 0.8925170068027211, 0.8802721088435375, 0.891156462585034]
Quantidade de classificadores: 15
Acurácia: 0.8966666666666667
DP: 0.005
[0.8918367346938776, 0.8877551020408163, 0.8931972789115646, 0.8918367346938776, 0.8972789115646258, 0.8993197278911564, 0.898639455782313, 0.9047619047619048, 0.9047619047619048, 0.8972789115646258]
Quantidade de classificadores: 20
Acurácia: 0.8895918367346939
DP: 0.010
[0.9136054421768708, 0.891156462585034, 0.9020408163265307, 0.8809523809523809, 0.8891156462585034, 0.8829931972789116, 0.8802721088435375, 0.882312925170068, 0.8802721088435375, 0.8931972789115646]


O modelo acima, utilizando o Naive como combinador, obteve melhor desempenho

In [ ]:
for tam in [10, 15, 20]:
  acuracia = []
  for x in range(10):
    seed = random.randint(900,5000)
    X = dataset.iloc[:,:-1].values
    y = dataset.iloc[:,-1].values

    rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=seed)
    for train, test in rkf.split(X, y):
      X_train = X[train]
      X_test = X[test]
      y_train = y[train]
      y_test = y[test]

    dtc = DecisionTreeClassifier(max_depth = 7, max_leaf_nodes = 19, min_samples_split = 2)

    if(tam == 10): 
      estimators = [('dtc1', dtc), ('dtc2', dtc), ('dtc3', dtc), ('dtc4', dtc), ('dtc5', dtc),
                    ('dtc6', dtc), ('dtc7', dtc), ('dtc8', dtc), ('dtc9', dtc), ('dtc10', dtc)]

    elif(tam == 15):
      estimators = [('dtc1', dtc), ('dtc2', dtc), ('dtc3', dtc), ('dtc4', dtc), ('dtc5', dtc),
                    ('dtc6', dtc), ('dtc7', dtc), ('dtc8', dtc), ('dtc9', dtc), ('dtc10', dtc),
                    ('dtc11', dtc), ('dtc12', dtc), ('dtc13', dtc), ('dtc14', dtc), ('dtc15', dtc)]
    
    elif(tam == 20):
      estimators = [('dtc1', dtc), ('dtc2', dtc), ('dtc3', dtc), ('dtc4', dtc), ('dtc5', dtc),
                    ('dtc6', dtc), ('dtc7', dtc), ('dtc8', dtc), ('dtc9', dtc), ('dtc10', dtc),
                    ('dtc11', dtc), ('dtc12', dtc), ('dtc13', dtc), ('dtc14', dtc), ('dtc15', dtc),
                    ('dtc16', dtc), ('dtc17', dtc), ('dtc18', dtc), ('dtc19', dtc), ('dtc20', dtc)]

    sclf = StackingClassifier(estimators=estimators, final_estimator=KNeighborsClassifier())
    sclf.fit(X_train, y_train)
    y_pred = sclf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    acuracia.append(acc)

  print('Quantidade de classificadores:', tam)
  print('Acurácia:', mean(acuracia))
  print('DP: %.3f' % std(acuracia))

Quantidade de classificadores: 10
Acurácia: 0.8835374149659865
DP: 0.020
Quantidade de classificadores: 15
Acurácia: 0.8827210884353741
DP: 0.010
Quantidade de classificadores: 20
Acurácia: 0.875578231292517
DP: 0.011


####**KNN**

In [ ]:
for tam in [10, 15, 20]:
  acuracia = []
  for x in range(10):
    seed = random.randint(900,5000)
    X = dataset.iloc[:,:-1].values
    y = dataset.iloc[:,-1].values

    rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=seed)
    for train, test in rkf.split(X, y):
      X_train = X[train]
      X_test = X[test]
      y_train = y[train]
      y_test = y[test]

    knn = KNeighborsClassifier(n_neighbors=7, metric='euclidean')

    if(tam == 10): 
      estimators = [('knn1', knn), ('knn2', knn), ('knn3', knn), ('knn4', knn), ('knn5', knn),
                    ('knn6', knn), ('knn7', knn), ('knn8', knn), ('knn9', knn), ('knn10', knn)]

    elif(tam == 15):
      estimators = [('knn1', knn), ('knn2', knn), ('knn3', knn), ('knn4', knn), ('knn5', knn),
                    ('knn6', knn), ('knn7', knn), ('knn8', knn), ('knn9', knn), ('knn10', knn),
                    ('knn11', knn), ('knn12', knn), ('knn13', knn), ('knn14', knn), ('knn15', knn)]
    
    elif(tam == 20):
      estimators = [('knn1', knn), ('knn2', knn), ('knn3', knn), ('knn4', knn), ('knn5', knn),
                    ('knn6', knn), ('knn7', knn), ('knn8', knn), ('knn9', knn), ('knn10', knn),
                    ('knn11', knn), ('knn12', knn), ('knn13', knn), ('knn14', knn), ('knn15', knn),
                    ('knn16', knn), ('knn17', knn), ('knn18', knn), ('knn19', knn), ('knn20', knn)]

    sclf = StackingClassifier(estimators=estimators, final_estimator=DecisionTreeClassifier())
    sclf.fit(X_train, y_train)
    y_pred = sclf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    acuracia.append(acc)

  print('Quantidade de classificadores:', tam)
  print('Acurácia:', mean(acuracia))
  print('DP: %.3f' % std(acuracia))

Quantidade de classificadores: 10
Acurácia: 0.6223129251700682
DP: 0.010
Quantidade de classificadores: 15
Acurácia: 0.6307482993197279
DP: 0.012
Quantidade de classificadores: 20
Acurácia: 0.6274149659863946
DP: 0.013


In [ ]:
for tam in [10, 15, 20]:
  acuracia = []
  for x in range(10):
    seed = random.randint(900,5000)
    X = dataset.iloc[:,:-1].values
    y = dataset.iloc[:,-1].values

    rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=seed)
    for train, test in rkf.split(X, y):
      X_train = X[train]
      X_test = X[test]
      y_train = y[train]
      y_test = y[test]

    knn = KNeighborsClassifier(n_neighbors=7, metric='euclidean')

    if(tam == 10): 
      estimators = [('knn1', knn), ('knn2', knn), ('knn3', knn), ('knn4', knn), ('knn5', knn),
                    ('knn6', knn), ('knn7', knn), ('knn8', knn), ('knn9', knn), ('knn10', knn)]

    elif(tam == 15):
      estimators = [('knn1', knn), ('knn2', knn), ('knn3', knn), ('knn4', knn), ('knn5', knn),
                    ('knn6', knn), ('knn7', knn), ('knn8', knn), ('knn9', knn), ('knn10', knn),
                    ('knn11', knn), ('knn12', knn), ('knn13', knn), ('knn14', knn), ('knn15', knn)]
    
    elif(tam == 20):
      estimators = [('knn1', knn), ('knn2', knn), ('knn3', knn), ('knn4', knn), ('knn5', knn),
                    ('knn6', knn), ('knn7', knn), ('knn8', knn), ('knn9', knn), ('knn10', knn),
                    ('knn11', knn), ('knn12', knn), ('knn13', knn), ('knn14', knn), ('knn15', knn),
                    ('knn16', knn), ('knn17', knn), ('knn18', knn), ('knn19', knn), ('knn20', knn)]

    sclf = StackingClassifier(estimators=estimators, final_estimator=KNeighborsClassifier())
    sclf.fit(X_train, y_train)
    y_pred = sclf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    acuracia.append(acc)

  print('Quantidade de classificadores:', tam)
  print('Acurácia:', mean(acuracia))
  print('DP: %.3f' % std(acuracia))

Quantidade de classificadores: 10
Acurácia: 0.5648299319727891
DP: 0.039
Quantidade de classificadores: 15
Acurácia: 0.587278911564626
DP: 0.047
Quantidade de classificadores: 20
Acurácia: 0.5612925170068027
DP: 0.019


In [ ]:
for tam in [10, 15, 20]:
  acuracia = []
  for x in range(10):
    seed = random.randint(900,5000)
    X = dataset.iloc[:,:-1].values
    y = dataset.iloc[:,-1].values

    rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=seed)
    for train, test in rkf.split(X, y):
      X_train = X[train]
      X_test = X[test]
      y_train = y[train]
      y_test = y[test]

    knn = KNeighborsClassifier(n_neighbors=7, metric='euclidean')

    if(tam == 10): 
      estimators = [('knn1', knn), ('knn2', knn), ('knn3', knn), ('knn4', knn), ('knn5', knn),
                    ('knn6', knn), ('knn7', knn), ('knn8', knn), ('knn9', knn), ('knn10', knn)]

    elif(tam == 15):
      estimators = [('knn1', knn), ('knn2', knn), ('knn3', knn), ('knn4', knn), ('knn5', knn),
                    ('knn6', knn), ('knn7', knn), ('knn8', knn), ('knn9', knn), ('knn10', knn),
                    ('knn11', knn), ('knn12', knn), ('knn13', knn), ('knn14', knn), ('knn15', knn)]
    
    elif(tam == 20):
      estimators = [('knn1', knn), ('knn2', knn), ('knn3', knn), ('knn4', knn), ('knn5', knn),
                    ('knn6', knn), ('knn7', knn), ('knn8', knn), ('knn9', knn), ('knn10', knn),
                    ('knn11', knn), ('knn12', knn), ('knn13', knn), ('knn14', knn), ('knn15', knn),
                    ('knn16', knn), ('knn17', knn), ('knn18', knn), ('knn19', knn), ('knn20', knn)]

    sclf = StackingClassifier(estimators=estimators, final_estimator=Naive())
    sclf.fit(X_train, y_train)
    y_pred = sclf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    acuracia.append(acc)

  print('Quantidade de classificadores:', tam)
  print('Acurácia:', mean(acuracia))
  print('DP: %.3f' % std(acuracia))
  print(acuracia)

Quantidade de classificadores: 10
Acurácia: 0.6302721088435373
DP: 0.010
[0.6142857142857143, 0.6333333333333333, 0.6346938775510204, 0.6299319727891156, 0.6374149659863946, 0.6448979591836734, 0.6258503401360545, 0.6353741496598639, 0.6340136054421769, 0.6129251700680272]
Quantidade de classificadores: 15
Acurácia: 0.622517006802721
DP: 0.015
[0.6074829931972789, 0.6285714285714286, 0.6312925170068027, 0.5993197278911565, 0.6224489795918368, 0.6435374149659864, 0.64421768707483, 0.6204081632653061, 0.6034013605442177, 0.6244897959183674]
Quantidade de classificadores: 20
Acurácia: 0.6263945578231294
DP: 0.012
[0.6190476190476191, 0.6251700680272109, 0.6340136054421769, 0.6108843537414966, 0.6469387755102041, 0.6394557823129252, 0.6142857142857143, 0.6346938775510204, 0.6299319727891156, 0.6095238095238096]


O modelo acima, utilizando o Naive como combinador, obteve melhor desempenho

####**Naive**

In [ ]:
for tam in [10, 15, 20]:
  acuracia = []
  for x in range(10):
    seed = random.randint(900,5000)
    X = dataset.iloc[:,:-1].values
    y = dataset.iloc[:,-1].values

    rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=seed)
    for train, test in rkf.split(X, y):
      X_train = X[train]
      X_test = X[test]
      y_train = y[train]
      y_test = y[test]

    nb = Naive()

    if(tam == 10): 
      estimators = [('nb1', nb), ('nb2', nb), ('nb3', nb), ('nb4', nb), ('nb5', nb),
                    ('nb6', nb), ('nb7', nb), ('nb8', nb), ('nb9', nb), ('nb10', nb)]

    elif(tam == 15):
      estimators = [('nb1', nb), ('nb2', nb), ('nb3', nb), ('nb4', nb), ('nb5', nb),
                    ('nb6', nb), ('nb7', nb), ('nb8', nb), ('nb9', nb), ('nb10', nb),
                    ('nb11', nb), ('nb12', nb), ('nb13', nb), ('nb14', nb), ('nb15', nb)]
    
    elif(tam == 20):
      estimators = [('nb1', nb), ('nb2', nb), ('nb3', nb), ('nb4', nb), ('nb5', nb),
                    ('nb6', nb), ('nb7', nb), ('nb8', nb), ('nb9', nb), ('nb10', nb),
                    ('nb11', nb), ('nb12', nb), ('nb13', nb), ('nb14', nb), ('nb15', nb),
                    ('nb16', nb), ('nb17', nb), ('nb18', nb), ('nb19', nb), ('nb20', nb)]

    sclf = StackingClassifier(estimators=estimators, final_estimator=DecisionTreeClassifier())
    sclf.fit(X_train, y_train)
    y_pred = sclf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    acuracia.append(acc)

  print('Quantidade de classificadores:', tam)
  print('Acurácia:', mean(acuracia))
  print('DP: %.3f' % std(acuracia))

Quantidade de classificadores: 10
Acurácia: 0.7521088435374149
DP: 0.010
Quantidade de classificadores: 15
Acurácia: 0.7561224489795919
DP: 0.010
Quantidade de classificadores: 20
Acurácia: 0.7589795918367347
DP: 0.011


In [ ]:
for tam in [10, 15, 20]:
  acuracia = []
  for x in range(10):
    seed = random.randint(900,5000)
    X = dataset.iloc[:,:-1].values
    y = dataset.iloc[:,-1].values

    rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=seed)
    for train, test in rkf.split(X, y):
      X_train = X[train]
      X_test = X[test]
      y_train = y[train]
      y_test = y[test]

    nb = Naive()

    if(tam == 10): 
      estimators = [('nb1', nb), ('nb2', nb), ('nb3', nb), ('nb4', nb), ('nb5', nb),
                    ('nb6', nb), ('nb7', nb), ('nb8', nb), ('nb9', nb), ('nb10', nb)]

    elif(tam == 15):
      estimators = [('nb1', nb), ('nb2', nb), ('nb3', nb), ('nb4', nb), ('nb5', nb),
                    ('nb6', nb), ('nb7', nb), ('nb8', nb), ('nb9', nb), ('nb10', nb),
                    ('nb11', nb), ('nb12', nb), ('nb13', nb), ('nb14', nb), ('nb15', nb)]
    
    elif(tam == 20):
      estimators = [('nb1', nb), ('nb2', nb), ('nb3', nb), ('nb4', nb), ('nb5', nb),
                    ('nb6', nb), ('nb7', nb), ('nb8', nb), ('nb9', nb), ('nb10', nb),
                    ('nb11', nb), ('nb12', nb), ('nb13', nb), ('nb14', nb), ('nb15', nb),
                    ('nb16', nb), ('nb17', nb), ('nb18', nb), ('nb19', nb), ('nb20', nb)]

    sclf = StackingClassifier(estimators=estimators, final_estimator=KNeighborsClassifier())
    sclf.fit(X_train, y_train)
    y_pred = sclf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    acuracia.append(acc)

  print('Quantidade de classificadores:', tam)
  print('Acurácia:', mean(acuracia))
  print('DP: %.3f' % std(acuracia))

Quantidade de classificadores: 10
Acurácia: 0.8061224489795918
DP: 0.012
Quantidade de classificadores: 15
Acurácia: 0.8012925170068028
DP: 0.007
Quantidade de classificadores: 20
Acurácia: 0.802721088435374
DP: 0.007


In [ ]:
for tam in [10, 15, 20]:
  acuracia = []
  for x in range(10):
    seed = random.randint(900,5000)
    X = dataset.iloc[:,:-1].values
    y = dataset.iloc[:,-1].values

    rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=seed)
    for train, test in rkf.split(X, y):
      X_train = X[train]
      X_test = X[test]
      y_train = y[train]
      y_test = y[test]

    nb = Naive()

    if(tam == 10): 
      estimators = [('nb1', nb), ('nb2', nb), ('nb3', nb), ('nb4', nb), ('nb5', nb),
                    ('nb6', nb), ('nb7', nb), ('nb8', nb), ('nb9', nb), ('nb10', nb)]

    elif(tam == 15):
      estimators = [('nb1', nb), ('nb2', nb), ('nb3', nb), ('nb4', nb), ('nb5', nb),
                    ('nb6', nb), ('nb7', nb), ('nb8', nb), ('nb9', nb), ('nb10', nb),
                    ('nb11', nb), ('nb12', nb), ('nb13', nb), ('nb14', nb), ('nb15', nb)]
    
    elif(tam == 20):
      estimators = [('nb1', nb), ('nb2', nb), ('nb3', nb), ('nb4', nb), ('nb5', nb),
                    ('nb6', nb), ('nb7', nb), ('nb8', nb), ('nb9', nb), ('nb10', nb),
                    ('nb11', nb), ('nb12', nb), ('nb13', nb), ('nb14', nb), ('nb15', nb),
                    ('nb16', nb), ('nb17', nb), ('nb18', nb), ('nb19', nb), ('nb20', nb)]

    sclf = StackingClassifier(estimators=estimators, final_estimator=Naive())
    sclf.fit(X_train, y_train)
    y_pred = sclf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    acuracia.append(acc)

  print('Quantidade de classificadores:', tam)
  print('Acurácia:', mean(acuracia))
  print('DP: %.3f' % std(acuracia))
  print(acuracia)

Quantidade de classificadores: 10
Acurácia: 0.8251700680272108
DP: 0.009
[0.8244897959183674, 0.827891156462585, 0.8258503401360544, 0.8387755102040816, 0.8231292517006803, 0.8224489795918367, 0.8251700680272109, 0.8034013605442177, 0.8299319727891157, 0.8306122448979592]
Quantidade de classificadores: 15
Acurácia: 0.8189115646258504
DP: 0.011
[0.8251700680272109, 0.8292517006802721, 0.808843537414966, 0.8333333333333334, 0.8231292517006803, 0.8068027210884354, 0.8020408163265306, 0.808843537414966, 0.8292517006802721, 0.8224489795918367]
Quantidade de classificadores: 20
Acurácia: 0.8181632653061225
DP: 0.010
[0.8095238095238095, 0.8244897959183674, 0.7938775510204081, 0.8224489795918367, 0.8142857142857143, 0.8217687074829932, 0.8204081632653061, 0.8333333333333334, 0.8163265306122449, 0.8251700680272109]


O modelo acima, utilizando o Naive como combinador, obteve melhor desempenho

####**MLP**

In [ ]:
for tam in [10, 15, 20]:
  acuracia = []
  for x in range(10):
    seed = random.randint(900,5000)
    X = dataset.iloc[:,:-1].values
    y = dataset.iloc[:,-1].values

    rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=seed)
    for train, test in rkf.split(X, y):
      X_train = X[train]
      X_test = X[test]
      y_train = y[train]
      y_test = y[test]

    mlp = MLPClassifier(solver='sgd', momentum=0.8, 
                            hidden_layer_sizes=16, 
                            learning_rate='constant', 
                            learning_rate_init=0.1,
                            max_iter=100, 
                            random_state=1)

    if(tam == 10): 
      estimators = [('mlp1', mlp), ('mlp2', mlp), ('mlp3', mlp), ('mlp4', mlp), ('mlp5', mlp),
                    ('mlp6', mlp), ('mlp7', mlp), ('mlp8', mlp), ('mlp9', mlp), ('mlp10', mlp)]

    elif(tam == 15):
      estimators = [('mlp1', mlp), ('mlp2', mlp), ('mlp3', mlp), ('mlp4', mlp), ('mlp5', mlp),
                    ('mlp6', mlp), ('mlp7', mlp), ('mlp8', mlp), ('mlp9', mlp), ('mlp10', mlp),
                    ('mlp11', mlp), ('mlp12', mlp), ('mlp13', mlp), ('mlp14', mlp), ('mlp15', mlp)]
    
    elif(tam == 20):
      estimators = [('mlp1', mlp), ('mlp2', mlp), ('mlp3', mlp), ('mlp4', mlp), ('mlp5', mlp),
                    ('mlp6', mlp), ('mlp7', mlp), ('mlp8', mlp), ('mlp9', mlp), ('mlp10', mlp),
                    ('mlp11', mlp), ('mlp12', mlp), ('mlp13', mlp), ('mlp14', mlp), ('mlp15', mlp),
                    ('mlp16', mlp), ('mlp17', mlp), ('mlp18', mlp), ('mlp19', mlp), ('mlp20', mlp)]

    sclf = StackingClassifier(estimators=estimators, final_estimator=DecisionTreeClassifier())
    sclf.fit(X_train, y_train)
    y_pred = sclf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    acuracia.append(acc)

  print('Quantidade de classificadores:', tam)
  print('Acurácia:', mean(acuracia))
  print('DP: %.3f' % std(acuracia))

* Quantidade de classificadores: 10
>* Acurácia: 0.5416326530612244
>* DP: 0.012

* Quantidade de classificadores: 15
>* Acurácia: 0.5423129251700681
>* DP: 0.011

* Quantidade de classificadores: 20
>* Acurácia: 0.5416326530612244
>* DP: 0.013

In [ ]:
for tam in [10, 15, 20]:
  acuracia = []
  for x in range(10):
    seed = random.randint(900,5000)
    X = dataset.iloc[:,:-1].values
    y = dataset.iloc[:,-1].values

    rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=seed)
    for train, test in rkf.split(X, y):
      X_train = X[train]
      X_test = X[test]
      y_train = y[train]
      y_test = y[test]

    mlp = MLPClassifier(solver='sgd', momentum=0.8, 
                            hidden_layer_sizes=16, 
                            learning_rate='constant', 
                            learning_rate_init=0.1,
                            max_iter=100, 
                            random_state=1)

    if(tam == 10): 
      estimators = [('mlp1', mlp), ('mlp2', mlp), ('mlp3', mlp), ('mlp4', mlp), ('mlp5', mlp),
                    ('mlp6', mlp), ('mlp7', mlp), ('mlp8', mlp), ('mlp9', mlp), ('mlp10', mlp)]

    elif(tam == 15):
      estimators = [('mlp1', mlp), ('mlp2', mlp), ('mlp3', mlp), ('mlp4', mlp), ('mlp5', mlp),
                    ('mlp6', mlp), ('mlp7', mlp), ('mlp8', mlp), ('mlp9', mlp), ('mlp10', mlp),
                    ('mlp11', mlp), ('mlp12', mlp), ('mlp13', mlp), ('mlp14', mlp), ('mlp15', mlp)]
    
    elif(tam == 20):
      estimators = [('mlp1', mlp), ('mlp2', mlp), ('mlp3', mlp), ('mlp4', mlp), ('mlp5', mlp),
                    ('mlp6', mlp), ('mlp7', mlp), ('mlp8', mlp), ('mlp9', mlp), ('mlp10', mlp),
                    ('mlp11', mlp), ('mlp12', mlp), ('mlp13', mlp), ('mlp14', mlp), ('mlp15', mlp),
                    ('mlp16', mlp), ('mlp17', mlp), ('mlp18', mlp), ('mlp19', mlp), ('mlp20', mlp)]

    sclf = StackingClassifier(estimators=estimators, final_estimator=Naive())
    sclf.fit(X_train, y_train)
    y_pred = sclf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    acuracia.append(acc)

  print('Quantidade de classificadores:', tam)
  print('Acurácia:', mean(acuracia))
  print('DP: %.3f' % std(acuracia))
  print(acuracia)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Quantidade de classificadores: 10
Acurácia: 0.4870748299319728
DP: 0.039
[0.5285714285714286, 0.4557823129251701, 0.5238095238095238, 0.4605442176870748, 0.545578231292517, 0.4748299319727891, 0.44625850340136053, 0.4496598639455782, 0.45102040816326533, 0.5346938775510204]


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Quantidade de classificadores: 15
Acurácia: 0.49482993197278907
DP: 0.039
[0.46122448979591835, 0.535374149659864, 0.5523809523809524, 0.45918367346938777, 0.46530612244897956, 0.45782312925170066, 0.527891156462585, 0.46530612244897956, 0.47346938775510206, 0.5503401360544218]


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Quantidade de classificadores: 20
Acurácia: 0.5146938775510204
DP: 0.036
[0.5292517006802722, 0.54421768707483, 0.5387755102040817, 0.4557823129251701, 0.46598639455782315, 0.5469387755102041, 0.45918367346938777, 0.5319727891156463, 0.5244897959183673, 0.5503401360544218]


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


* Quantidade de classificadores: 10
>* Acurácia: 0.5152380952380953
>* DP: 0.041

* Quantidade de classificadores: 15
>* Acurácia: 0.5009523809523809
>* DP: 0.036

* Quantidade de classificadores: 20
>* Acurácia: 0.48802721088435375
>* DP: 0.031

### **Stacking Heterogêneo**

Para criar a tabela do Stacking heterogêneo, utilize os mesmos tamanhos das configurações homogêneas (10, 15 e 20 classificadores). Para os algoritmos de classificação, escolha três classificadores base, de acordo com o desempenho dos classificadores individuais. Por exemplo, se na sua análise comparativa, o melhor classificador foi MLP, seguido por: k-NN, AD e NB, então escolha o MLP (Método A), k-NN (Método B) e a AD (Método C), pois estes são os três classificadores com o primeiro, segundo e terceiro melhor desempenho. Então, utilize quatro configurações:
* A. 50% do Método A e 50% do Método B
* B. 50% do Método A e 50% do Método C
* C. 50% do Método B e 50% do Método C
* D. 33% do Método A, 33% do Método B e 33% do Método C

Nos resultados individuais (relatório do Checkpoint 2)
* 1º MLP = 0.9285714
* 2º KNN = 0.9144898
* 3º AD = 0.8979592


####**Configuração A - 50% do Método A e 50% do Método B**

In [ ]:
for tam in [10, 15, 20]:
  acuracia = []
  for x in range(10):
    seed = random.randint(900,5000)
    X = dataset.iloc[:,:-1].values
    y = dataset.iloc[:,-1].values

    rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=seed)
    for train, test in rkf.split(X, y):
      X_train = X[train]
      X_test = X[test]
      y_train = y[train]
      y_test = y[test]

    mlp = MLPClassifier(solver='sgd', momentum=0.8, 
                                hidden_layer_sizes=16, 
                                learning_rate='constant', 
                                learning_rate_init=0.1,
                                max_iter=100, 
                                random_state=1)

    knn = KNeighborsClassifier(n_neighbors=7, metric='euclidean')

    if(tam == 10): 
      estimators = [('knn1', knn), ('knn2', knn), ('knn3', knn), ('knn4', knn), ('knn5', knn),
                    ('mlp1', mlp), ('mlp2', mlp), ('mlp3', mlp), ('mlp4', mlp), ('mlp5', mlp)]

    elif(tam == 15):
      estimators = [('knn1', knn), ('knn2', knn), ('knn3', knn), ('knn4', knn), ('knn5', knn),
                    ('knn6', knn), ('knn7', knn), ('knn8', knn), ('mlp9', mlp), ('mlp10', mlp),
                    ('mlp11', mlp), ('mlp12', mlp), ('mlp13', mlp), ('mlp14', mlp), ('mlp15', mlp)]

    elif(tam == 20):
      estimators = [('knn1', knn), ('knn2', knn), ('knn3', knn), ('knn4', knn), ('knn5', knn),
                    ('knn6', knn), ('knn7', knn), ('knn8', knn), ('knn9', knn), ('knn10', knn),
                    ('mlp1', mlp), ('mlp2', mlp), ('mlp3', mlp), ('mlp4', mlp), ('mlp5', mlp),
                    ('mlp6', mlp), ('mlp7', mlp), ('mlp8', mlp), ('mlp9', mlp), ('mlp10', mlp)] 

    sclf = StackingClassifier(estimators=estimators, final_estimator=Naive())
    sclf.fit(X_train, y_train)
    y_pred = sclf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    acuracia.append(acc)

  print('Quantidade de classificadores:', tam)
  print('Acurácia:', mean(acuracia))
  print('DP: %.3f' % std(acuracia))
  print(acuracia)

**Resultados:**

* Quantidade de classificadores: 10
>* Acurácia: 0.5604761904761905
>* DP: 0.076

* Quantidade de classificadores: 15
>* Acurácia: 0.5557142857142857
>* DP: 0.085

* Quantidade de classificadores: 20
>* Acurácia: 0.5100680272108843
>* DP: 0.069

####**Configuração B - 50% do Método A e 50% do Método C**

In [ ]:
for tam in [10, 15, 20]:
  acuracia = []
  for x in range(10):
    seed = random.randint(900,5000)
    X = dataset.iloc[:,:-1].values
    y = dataset.iloc[:,-1].values

    rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=seed)
    for train, test in rkf.split(X, y):
      X_train = X[train]
      X_test = X[test]
      y_train = y[train]
      y_test = y[test]

    mlp = MLPClassifier(solver='sgd', momentum=0.8, 
                                hidden_layer_sizes=16, 
                                learning_rate='constant', 
                                learning_rate_init=0.1,
                                max_iter=100, 
                                random_state=1)

    dtc = DecisionTreeClassifier(max_depth = 7, max_leaf_nodes = 19, min_samples_split = 2)

    if(tam == 10): 
      estimators = [('dtc1', dtc), ('dtc2', dtc), ('dtc3', dtc), ('dtc4', dtc), ('dtc5', dtc),
                    ('mlp1', mlp), ('mlp2', mlp), ('mlp3', mlp), ('mlp4', mlp), ('mlp5', mlp)]

    elif(tam == 15):
      estimators = [('dtc1', dtc), ('dtc2', dtc), ('dtc3', dtc), ('dtc4', dtc), ('dtc5', dtc),
                    ('dtc6', dtc), ('dtc7', dtc), ('dtc8', dtc), ('mlp9', mlp), ('mlp10', mlp),
                    ('mlp11', mlp), ('mlp12', mlp), ('mlp13', mlp), ('mlp14', mlp), ('mlp15', mlp)]

    elif(tam == 20):
      estimators = [('dtc1', dtc), ('dtc2', dtc), ('dtc3', dtc), ('dtc4', dtc), ('dtc5', dtc),
                    ('dtc6', dtc), ('dtc7', dtc), ('dtc8', dtc), ('dtc9', dtc), ('dtc10', dtc),
                    ('mlp1', mlp), ('mlp2', mlp), ('mlp3', mlp), ('mlp4', mlp), ('mlp5', mlp),
                    ('mlp6', mlp), ('mlp7', mlp), ('mlp8', mlp), ('mlp9', mlp), ('mlp10', mlp)] 

    sclf = StackingClassifier(estimators=estimators, final_estimator=Naive())
    sclf.fit(X_train, y_train)
    y_pred = sclf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    acuracia.append(acc)

  print('Quantidade de classificadores:', tam)
  print('Acurácia:', mean(acuracia))
  print('DP: %.3f' % std(acuracia))
  print(acuracia)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Quantidade de classificadores: 10
Acurácia: 0.7631972789115646
DP: 0.190


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Quantidade de classificadores: 15
Acurácia: 0.8446938775510204
DP: 0.135


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Quantidade de classificadores: 20
Acurácia: 0.8485714285714285
DP: 0.124


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


**Resultados:**

* Quantidade de classificadores: 10
>* Acurácia: 0.7631972789115646
>* DP: 0.190

* Quantidade de classificadores: 15
>* Acurácia: 0.8446938775510204
>* DP: 0.135

* Quantidade de classificadores: 20
>* Acurácia: 0.8485714285714285
>* DP: 0.124

####**Configuração C - 50% do Método B e 50% do Método C**

In [ ]:
for tam in [10, 15, 20]:
  acuracia = []
  for x in range(10):
    seed = random.randint(900,5000)
    X = dataset.iloc[:,:-1].values
    y = dataset.iloc[:,-1].values

    rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=seed)
    for train, test in rkf.split(X, y):
      X_train = X[train]
      X_test = X[test]
      y_train = y[train]
      y_test = y[test]

    knn = KNeighborsClassifier(n_neighbors=7, metric='euclidean')
    dtc = DecisionTreeClassifier(max_depth = 7, max_leaf_nodes = 19, min_samples_split = 2)

    if(tam == 10): 
      estimators = [('knn1', knn), ('knn2', knn), ('knn3', knn), ('knn4', knn), ('knn5', knn),
                    ('dtc1', dtc), ('dtc2', dtc), ('dtc3', dtc), ('dtc4', dtc), ('dtc5', dtc)]

    elif(tam == 15):
      estimators = [('knn1', knn), ('knn2', knn), ('knn3', knn), ('knn4', knn), ('knn5', knn),
                    ('knn6', knn), ('knn7', knn), ('knn8', knn), ('dtc9', dtc), ('dtc10', dtc),
                    ('dtc11', dtc), ('dtc12', dtc), ('dtc13', dtc), ('dtc14', dtc), ('dtc15', dtc)]

    elif(tam == 20):
      estimators = [('knn1', knn), ('knn2', knn), ('knn3', knn), ('knn4', knn), ('knn5', knn),
                    ('knn6', knn), ('knn7', knn), ('knn8', knn), ('knn9', knn), ('knn10', knn),
                    ('dtc1', dtc), ('dtc2', dtc), ('dtc3', dtc), ('dtc4', dtc), ('dtc5', dtc),
                    ('dtc6', dtc), ('dtc7', dtc), ('dtc8', dtc), ('dtc9', dtc), ('dtc10', dtc)] 

    sclf = StackingClassifier(estimators=estimators, final_estimator=Naive())
    sclf.fit(X_train, y_train)
    y_pred = sclf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    acuracia.append(acc)

  print('Quantidade de classificadores:', tam)
  print('Acurácia:', mean(acuracia))
  print('DP: %.3f' % std(acuracia))
  print(acuracia)

Quantidade de classificadores: 10
Acurácia: 0.8953741496598641
DP: 0.010
Quantidade de classificadores: 15
Acurácia: 0.8971428571428571
DP: 0.009
Quantidade de classificadores: 20
Acurácia: 0.8961904761904762
DP: 0.012


####**Configuração D - 33% do Método A, 33% do Método B e 33% do Método C**

In [ ]:
for tam in [10, 15, 20]:
  acuracia = []
  for x in range(10):
    seed = random.randint(900,5000)
    X = dataset.iloc[:,:-1].values
    y = dataset.iloc[:,-1].values

    rkf = RepeatedKFold(n_splits=10, n_repeats=10, random_state=seed)
    for train, test in rkf.split(X, y):
      X_train = X[train]
      X_test = X[test]
      y_train = y[train]
      y_test = y[test]

    mlp = MLPClassifier(solver='sgd', momentum=0.8, 
                                hidden_layer_sizes=16, 
                                learning_rate='constant', 
                                learning_rate_init=0.1,
                                max_iter=100, 
                                random_state=1)
    
    knn = KNeighborsClassifier(n_neighbors=7, metric='euclidean')

    dtc = DecisionTreeClassifier(max_depth = 7, max_leaf_nodes = 19, min_samples_split = 2)

    if(tam == 10): 
      estimators = [('mlp1', mlp), ('mlp2', mlp), ('mlp3', mlp), ('mlp4', mlp), 
                    ('knn1', knn), ('knn2', knn), ('knn3', knn), 
                    ('dtc1', dtc), ('dtc2', dtc), ('dtc3', dtc)]

    elif(tam == 15):
      estimators = [('mlp1', mlp), ('mlp2', mlp), ('mlp3', mlp), ('mlp4', mlp), ('mlp5', mlp),
                    ('knn1', knn), ('knn2', knn), ('knn3', knn), ('knn4', knn), ('knn5', knn),
                    ('dtc1', dtc), ('dtc2', dtc), ('dtc3', dtc), ('dtc4', dtc), ('dtc5', dtc)]

    elif(tam == 20):
      estimators = [('mlp1', mlp), ('mlp2', mlp), ('mlp3', mlp), ('mlp4', mlp), ('mlp5', mlp), ('mlp6', mlp), ('mlp7', mlp), 
                    ('knn1', knn), ('knn2', knn), ('knn3', knn), ('knn4', knn), ('knn5', knn), ('knn6', knn), ('knn7', knn), 
                    ('dtc1', dtc), ('dtc2', dtc), ('dtc3', dtc), ('dtc4', dtc), ('dtc5', dtc), ('dtc6', dtc)] 

    sclf = StackingClassifier(estimators=estimators, final_estimator=Naive())
    sclf.fit(X_train, y_train)
    y_pred = sclf.predict(X_test)

    acc = accuracy_score(y_test, y_pred)
    acuracia.append(acc)

  print('Quantidade de classificadores:', tam)
  print('Acurácia:', mean(acuracia))
  print('DP: %.3f' % std(acuracia))
  print(acuracia)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Quantidade de classificadores: 10
Acurácia: 0.854829931972789
DP: 0.116


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Quantidade de classificadores: 15
Acurácia: 0.892108843537415
DP: 0.008


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Quantidade de classificadores: 20
Acurácia: 0.8516326530612245
DP: 0.123


**Resultados:**

* Quantidade de classificadores: 10
>* Acurácia: 0.854829931972789
>* DP: 0.116

* Quantidade de classificadores: 15
>* Acurácia: 0.892108843537415
>* DP: 0.008

* Quantidade de classificadores: 20
>* Acurácia: 0.8516326530612245
>* DP: 0.123

###**Testes Estatísticos**

####**Nemenyi**

In [9]:
pip install scikit-posthocs

     |████████████████████████████████| 51kB 2.5MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for scikit-posthocs: filename=scikit_posthocs-0.6.7-cp37-none-any.whl size=37886 sha256=29b975ccf7f15e987d83602f3602ee681a66c81ecaa8a822868e98c8c0db01cd
  Stored in directory: /root/.cache/pip/wheels/00/0f/56/897c22820a48a74f50f1a85faaa4b133bb8865e8b094b596f5
Successfully built scikit-posthocs


In [15]:
import scikit_posthocs as sp

#####**Boosting**

In [10]:
bs10_ad = [0.9333333333333333, 0.9455782312925171, 0.9360544217687075, 0.9333333333333333, 0.9346938775510204, 0.9442176870748299, 0.9272108843537415, 0.9299319727891157, 0.9251700680272109, 0.9319727891156463]
bs10_knn = [0.9041468388851122, 0.8905506458191705, 0.9027872195785178, 0.9048266485384093, 0.91162474507138, 0.8966689326988444, 0.9123045547246772, 0.8932698844323589, 0.8966689326988444, 0.9014276002719238]
bs10_nb = [0.5761904761904761, 0.45374149659863944, 0.5231292517006803, 0.5448979591836735, 0.49183673469387756, 0.43673469387755104, 0.5210884353741496, 0.573469387755102, 0.4857142857142857, 0.4884353741496599]
bs10_mlp = [0.9380952380952381, 0.9326530612244898, 0.9326530612244898, 0.9489795918367347, 0.9380952380952381, 0.9360544217687075, 0.9333333333333333, 0.9401360544217687, 0.9312925170068027, 0.9387755102040817]

In [11]:
bs15_ad = [0.9285714285714286, 0.9421768707482994, 0.926530612244898, 0.9340136054421768, 0.9435374149659864, 0.9292517006802721, 0.9455782312925171, 0.9448979591836735, 0.9346938775510204, 0.9442176870748299]
bs15_knn = [0.8973487423521413, 0.8973487423521413, 0.8905506458191705, 0.8993881713120326, 0.8987083616587357, 0.9000679809653299, 0.9075458871515976, 0.8912304554724677, 0.9082256968048948, 0.9089055064581919]
bs15_nb = [0.5510204081632653, 0.7448979591836735, 0.6625850340136055, 0.6755102040816326, 0.6843537414965987, 0.5326530612244897, 0.6476190476190476, 0.47959183673469385, 0.582312925170068, 0.43741496598639457]
bs15_mlp = [0.9530612244897959, 0.9510204081632653, 0.9340136054421768, 0.9387755102040817, 0.935374149659864, 0.9394557823129251, 0.9224489795918367, 0.9421768707482994, 0.9469387755102041, 0.9448979591836735]

In [12]:
bs20_ad =  [0.9319727891156463, 0.9346938775510204, 0.9435374149659864, 0.9455782312925171, 0.9428571428571428, 0.9476190476190476, 0.9387755102040817, 0.9414965986394558, 0.9462585034013605, 0.9442176870748299]
bs20_knn = [0.8891910265125764, 0.8912304554724677, 0.9027872195785178, 0.9068660774983006, 0.8925900747790619, 0.8885112168592795, 0.9075458871515976, 0.9068660774983006, 0.9075458871515976, 0.8959183673469387]
bs20_nb =  [0.5727891156462585, 0.5448979591836735, 0.4523809523809524, 0.591156462585034,  0.427891156462585,  0.6632653061224489, 0.507482993197279,  0.5115646258503401, 0.5768707482993197, 0.4217687074829932]
bs20_mlp = [0.9476190476190476, 0.9435374149659864, 0.936734693877551,  0.9448979591836735, 0.9258503401360544, 0.9319727891156463, 0.9387755102040817, 0.9394557823129251, 0.9401360544217687, 0.9319727891156463]

In [16]:
data = np.array([bs10_ad, bs10_knn, bs10_nb, bs10_mlp])
sp.posthoc_nemenyi_friedman(data.T)

,0,1,2,3
0,1.000000,0.109694,0.00100,0.896267
1,0.109694,1.000000,0.30713,0.017062
2,0.001000,0.307130,1.00000,0.001000
3,0.896267,0.017062,0.00100,1.000000


In [19]:
data = np.array([bs15_ad, bs15_knn, bs15_nb, bs15_mlp])
sp.posthoc_nemenyi_friedman(data.T)

,0,1,2,3
0,1.000000,0.109694,0.00100,0.896267
1,0.109694,1.000000,0.30713,0.017062
2,0.001000,0.307130,1.00000,0.001000
3,0.896267,0.017062,0.00100,1.000000


In [20]:
data = np.array([bs20_ad, bs20_knn, bs20_nb, bs20_mlp])
sp.posthoc_nemenyi_friedman(data.T)

,0,1,2,3
0,1.000000,0.013021,0.00100,0.799047
1,0.013021,1.000000,0.30713,0.132924
2,0.001000,0.307130,1.00000,0.001000
3,0.799047,0.132924,0.00100,1.000000


#####**Bagging**

In [21]:
bg10_ad = [0.9333333333333333, 0.9455782312925171, 0.9360544217687075, 0.9333333333333333, 0.9346938775510204, 0.9442176870748299, 0.9272108843537415, 0.9299319727891157, 0.9251700680272109, 0.9319727891156463]
bg10_knn = [0.9047619047619048, 0.9108843537414966, 0.9081632653061225, 0.9163265306122449, 0.9081632653061225, 0.9081632653061225, 0.9068027210884354, 0.9102040816326531, 0.9054421768707483, 0.9142857142857143]
bg10_nb = [0.8149659863945579, 0.8238095238095238, 0.8142857142857143, 0.8210884353741497, 0.8156462585034013, 0.8353741496598639, 0.8272108843537415, 0.8197278911564626, 0.8170068027210884, 0.8272108843537415]
bg10_mlp = [0.9326530612244898, 0.9346938775510204, 0.935374149659864, 0.936734693877551, 0.9231292517006803, 0.926530612244898, 0.9380952380952381, 0.9217687074829932, 0.9224489795918367, 0.9258503401360544]

In [22]:
bg15_ad = [0.9285714285714286, 0.9421768707482994, 0.926530612244898, 0.9340136054421768, 0.9435374149659864, 0.9292517006802721, 0.9455782312925171, 0.9448979591836735, 0.9346938775510204, 0.94421768707482993]
bg15_knn = [0.917687074829932, 0.9108843537414966, 0.9163265306122449, 0.9244897959183673, 0.9122448979591836, 0.917687074829932, 0.9108843537414966, 0.9136054421768708, 0.9102040816326531, 0.9027210884353741]
bg15_nb = [0.826530612244898, 0.8312925170068027, 0.8306122448979592, 0.7979591836734694, 0.8163265306122449, 0.8047619047619048, 0.8197278911564626, 0.8129251700680272, 0.827891156462585, 0.827891156462585]
bg15_mlp = [0.9258503401360544, 0.927891156462585, 0.9340136054421768, 0.935374149659864, 0.9285714285714286, 0.9387755102040817, 0.9326530612244898, 0.9272108843537415, 0.9319727891156463, 0.927891156462585]

In [23]:
bg20_ad = [0.9319727891156463, 0.9346938775510204, 0.9435374149659864, 0.9455782312925171, 0.9428571428571428, 0.9476190476190476, 0.9387755102040817, 0.9414965986394558, 0.9462585034013605, 0.9442176870748299]
bg20_knn = [0.919047619047619, 0.919047619047619, 0.9217687074829932, 0.9149659863945578, 0.9210884353741496, 0.9149659863945578, 0.8965986394557823, 0.9142857142857143, 0.9163265306122449, 0.9054421768707483]
bg20_nb = [0.8013605442176871, 0.8204081632653061, 0.826530612244898, 0.8156462585034013, 0.8306122448979592, 0.7993197278911565, 0.8299319727891157, 0.8061224489795918, 0.8183673469387756, 0.8353741496598639]
bg20_mlp = [0.9312925170068027, 0.9183673469387755, 0.9374149659863945, 0.9319727891156463, 0.9374149659863945, 0.9394557823129251, 0.9380952380952381, 0.9319727891156463, 0.9306122448979591, 0.9374149659863945]

In [24]:
data = np.array([bg10_ad, bg10_knn, bg10_nb, bg10_mlp])
sp.posthoc_nemenyi_friedman(data.T)

,0,1,2,3
0,1.000000,0.009860,0.00100,0.701825
1,0.009860,1.000000,0.30713,0.160247
2,0.001000,0.307130,1.00000,0.001000
3,0.701825,0.160247,0.00100,1.000000


In [25]:
data = np.array([bg15_ad, bg15_knn, bg15_nb, bg15_mlp])
sp.posthoc_nemenyi_friedman(data.T)

,0,1,2,3
0,1.000000,0.017062,0.00100,0.896267
1,0.017062,1.000000,0.30713,0.109694
2,0.001000,0.307130,1.00000,0.001000
3,0.896267,0.109694,0.00100,1.000000


In [26]:
data = np.array([bg20_ad, bg20_knn, bg20_nb, bg20_mlp])
sp.posthoc_nemenyi_friedman(data.T)

,0,1,2,3
0,1.000000,0.005517,0.001000,0.225871
1,0.005517,1.000000,0.225871,0.507386
2,0.001000,0.225871,1.000000,0.005517
3,0.225871,0.507386,0.005517,1.000000


#####**Stacking Homogêneo**

In [27]:
hom10_ad = [0.8863945578231293, 0.8918367346938776, 0.8952380952380953, 0.9061224489795918, 0.8884353741496599, 0.898639455782313, 0.9020408163265307, 0.8925170068027211, 0.8802721088435375, 0.891156462585034]
hom10_knn = [0.6142857142857143, 0.6333333333333333, 0.6346938775510204, 0.6299319727891156, 0.6374149659863946, 0.6448979591836734, 0.6258503401360545, 0.6353741496598639, 0.6340136054421769, 0.6129251700680272]
hom10_nb = [0.8244897959183674, 0.827891156462585, 0.8258503401360544, 0.8387755102040816, 0.8231292517006803, 0.8224489795918367, 0.8251700680272109, 0.8034013605442177, 0.8299319727891157, 0.8306122448979592]
hom10_mlp = [0.5285714285714286, 0.4557823129251701, 0.5238095238095238, 0.4605442176870748, 0.545578231292517, 0.4748299319727891, 0.44625850340136053, 0.4496598639455782, 0.45102040816326533, 0.5346938775510204]

In [28]:
hom15_ad = [0.8918367346938776, 0.8877551020408163, 0.8931972789115646, 0.8918367346938776, 0.8972789115646258, 0.8993197278911564, 0.898639455782313, 0.9047619047619048, 0.9047619047619048, 0.8972789115646258]
hom15_knn = [0.6074829931972789, 0.6285714285714286, 0.6312925170068027, 0.5993197278911565, 0.6224489795918368, 0.6435374149659864, 0.64421768707483, 0.6204081632653061, 0.6034013605442177, 0.6244897959183674]
hom15_nb = [0.8251700680272109, 0.8292517006802721, 0.808843537414966, 0.8333333333333334, 0.8231292517006803, 0.8068027210884354, 0.8020408163265306, 0.808843537414966, 0.8292517006802721, 0.8224489795918367]
hom15_mlp = [0.5292517006802722, 0.54421768707483, 0.5387755102040817, 0.4557823129251701, 0.46598639455782315, 0.5469387755102041, 0.45918367346938777, 0.5319727891156463, 0.5244897959183673, 0.5503401360544218]

In [29]:
hom20_ad = [0.9136054421768708, 0.891156462585034, 0.9020408163265307, 0.8809523809523809, 0.8891156462585034, 0.8829931972789116, 0.8802721088435375, 0.882312925170068, 0.8802721088435375, 0.8931972789115646]
hom20_knn = [0.6190476190476191, 0.6251700680272109, 0.6340136054421769, 0.6108843537414966, 0.6469387755102041, 0.6394557823129252, 0.6142857142857143, 0.6346938775510204, 0.6299319727891156, 0.6095238095238096]
hom20_nb = [0.8095238095238095, 0.8244897959183674, 0.7938775510204081, 0.8224489795918367, 0.8142857142857143, 0.8217687074829932, 0.8204081632653061, 0.8333333333333334, 0.8163265306122449, 0.8251700680272109]
hom20_mlp = [0.5292517006802722, 0.54421768707483, 0.5387755102040817, 0.4557823129251701, 0.46598639455782315, 0.5469387755102041, 0.45918367346938777, 0.5319727891156463, 0.5244897959183673, 0.5503401360544218]

In [40]:
data = np.array([hom10_ad, hom10_knn, hom10_nb, hom10_mlp])
sp.posthoc_nemenyi_friedman(data.T)

,0,1,2,3
0,1.00000,0.00299,0.30713,0.00100
1,0.00299,1.00000,0.30713,0.30713
2,0.30713,0.30713,1.00000,0.00299
3,0.00100,0.30713,0.00299,1.00000


In [41]:
data = np.array([hom15_ad, hom15_knn, hom15_nb, hom15_mlp])
sp.posthoc_nemenyi_friedman(data.T)

,0,1,2,3
0,1.00000,0.00299,0.30713,0.00100
1,0.00299,1.00000,0.30713,0.30713
2,0.30713,0.30713,1.00000,0.00299
3,0.00100,0.30713,0.00299,1.00000


In [32]:
data = np.array([hom20_ad, hom20_knn, hom20_nb, hom20_mlp])
sp.posthoc_nemenyi_friedman(data.T)

,0,1,2,3
0,1.00000,0.00299,0.30713,0.00100
1,0.00299,1.00000,0.30713,0.30713
2,0.30713,0.30713,1.00000,0.00299
3,0.00100,0.30713,0.00299,1.00000


#####**Stacking Heterogêneo**

In [38]:
het10_a = [0.6149659863945578, 0.6265306122448979, 0.6346938775510204, 0.6183673469387755, 0.6272108843537415, 0.6231292517006802, 0.5401360544217687, 0.6047619047619047, 0.6340136054421769, 0.5829931972789115]
het10_b = [0.45850340136054424, 0.9108843537414966, 0.908843537414966, 0.8959183673469387, 0.9040816326530612, 0.8945578231292517, 0.9020408163265307, 0.9020408163265307, 0.47891156462585033, 0.8816326530612245]
het10_c = [0.9027210884353741, 0.8938775510204081, 0.8857142857142857, 0.8979591836734694, 0.898639455782313, 0.8959183673469387, 0.8925170068027211, 0.8972789115646258, 0.8979591836734694, 0.8925170068027211]
het10_d = [0.5816326530612245, 0.8891156462585034, 0.8945578231292517, 0.8829931972789116, 0.8925170068027211, 0.9020408163265307, 0.8782312925170068, 0.9040816326530612, 0.8918367346938776, 0.8857142857142857]

In [39]:
data = np.array([het10_a, het10_b, het10_c, het10_d])
sp.posthoc_nemenyi_friedman(data.T)

,0,1,2,3
0,1.000000,0.046280,0.005517,0.072567
1,0.046280,1.000000,0.896267,0.900000
2,0.005517,0.896267,1.000000,0.799047
3,0.072567,0.900000,0.799047,1.000000
